In [1]:
## Imports
import pandas as pd
from datasets import Dataset, DatasetDict
from transformers import AutoTokenizer, DataCollatorWithPadding
from transformers import Trainer, TrainingArguments
from transformers import AutoModelForSequenceClassification
from datasets import load_metric
import numpy as np

2022-03-28 13:09:02.169790: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-03-28 13:09:02.169893: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
## Data
filename = 'votes_data_cleaned_V2.pkl'
df = pd.read_pickle("../../data_collection/"+filename) #data_collection/votes_data_cleaned.pkl

test_set = Dataset.from_pandas(df, preserve_index=False)

test_set

Dataset({
    features: ['vote_id', 'description', 'votes', 'votes_by_party', 'vote_caller', 'url', 'description_text', 'date'],
    num_rows: 5559
})

In [4]:
## Load (part1)
from transformers import Trainer, TrainingArguments
from transformers import AutoModelForSequenceClassification
from datasets import load_metric

def compute_metrics(eval_preds):
    metric = load_metric("glue", "mrpc")
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

checkpoint = "/home/gugy/.cache/huggingface/transformers/klimaBERTe11_v2"

tokenizer = AutoTokenizer.from_pretrained(checkpoint, local_files_only=True)
training_args = TrainingArguments("test-trainer", evaluation_strategy="epoch")
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2, local_files_only=True)

## Load trainer API
trainer = Trainer(
    model,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)


Didn't find file /home/gugy/.cache/huggingface/transformers/klimaBERTe11_v2/added_tokens.json. We won't load it.
loading file /home/gugy/.cache/huggingface/transformers/klimaBERTe11_v2/vocab.txt
loading file /home/gugy/.cache/huggingface/transformers/klimaBERTe11_v2/tokenizer.json
loading file None
loading file /home/gugy/.cache/huggingface/transformers/klimaBERTe11_v2/special_tokens_map.json
loading file /home/gugy/.cache/huggingface/transformers/klimaBERTe11_v2/tokenizer_config.json
PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
loading configuration file /home/gugy/.cache/huggingface/transformers/klimaBERTe11_v2/config.json
Model config BertConfig {
  "_name_or_path": "/home/gugy/.cache/huggingface/transformers/klimaBERTe11_v2",
  "archit

In [5]:
## Set to use GPU
import torch

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)
device

device(type='cpu')

In [6]:
## Load FT data and tokenize it
def tokenize(batch):
    tokenized_batch = tokenizer(batch['description_text'], padding=True, truncation=True, max_length=512)
    return tokenized_batch

dataset = test_set
tokenized_datasets = dataset.map(tokenize, batched=True)
tokenized_datasets.set_format("torch")

  0%|          | 0/6 [00:00<?, ?ba/s]

In [ ]:
## Predict on sub-set
predictions2 = trainer.predict(tokenized_datasets)
predictions2

In [8]:
label = []

for (i,j) in predictions2[0]:
  if i > j: label.append("non-climate")
  else: label.append("climate")

In [9]:
## Compare y_real with y_pred
df_compare = pd.DataFrame()
df_compare = dataset.to_pandas()
df_compare["y_pred"] = label

In [10]:
df_climate = df_compare[df_compare["y_pred"]=="climate"].reset_index()
df_non_climate = df_compare[df_compare["y_pred"]=="non-climate"].reset_index()
print("y_pred:climate",len(df_climate.index))
print("y_pred:non-climate",len(df_non_climate.index))

# df_climate.to_csv("votes_data_climate.csv")
# df_non_climate.to_csv("votes_data_non_climate.csv")

df_climate.to_pickle("votes_data_climate.pkl")
df_non_climate.to_pickle("votes_data_non_climate.pkl")

y_pred:climate 236
y_pred:non-climate 8432


In [3]:
climate_votes = pd.read_pickle("votes_data_climate.pkl")
no_climate_votes = pd.read_pickle("votes_data_non_climate.pkl")


In [4]:
no_climate_votes.dropna(inplace=True)

In [5]:
climate_votes.drop_duplicates(subset='description_text', inplace=True)

climate_votes.reset_index(drop=True, inplace=True)



In [6]:
climate_votes['date']

0      2004-12-07
1      2005-05-17
2      2005-05-26
3      2005-06-01
4      2005-12-21
          ...    
196    2021-12-28
197    2022-03-03
198    2022-03-03
199    2022-03-03
200    2022-03-03
Name: date, Length: 201, dtype: object

In [7]:
climate_votes['description_text'][15]

"Beslutningsforslaget opfordrede finansministeren til at trække aktstykke nr. 53 om salg af aktier i DONG Energy A/S tilbage. Ifølge forslagsstillerne var der flere gode grunde til at revurdere det fremlagte aktstykke, bl.a. et usædvanligt forslag om, at ledelsen, banker og rådgivere i forbindelse med salget fritages for ansvar. Endvidere var der en række verserende sager mod DONG fra kunder og andre aktionærer i DONG, og fra en klimapolitisk synsvinkel var det efter forslagsstillernes opfattelse forkert at privatisere landets største energiproducent. En aktuel sag med DONG Energy's byggeri af et kulkraftværk i Tyskland viste de negative klimapolitiske konsekvenser ved at drive selskabet efter økonomiske og ikke klimamæssige hensyn."

In [8]:
import re

klimaforandringer = re.compile(r"(K|k)lima(forandringer)?(ne)?")
co2 = re.compile(r"(CO 2)|(CO2)|(kulstof(udledning(en)?)?)|(co2)|drivhusgas(ser)?|klimaplan")
grøn_omstilling = re.compile(r"grøn(ne)? omstilling")
global_opvarmning = re.compile(r"global(e)? opvarmning")


In [9]:
no_climate_votes['klimaforandringer'] = no_climate_votes['description_text'].apply(lambda x: 1 if re.search(klimaforandringer, x)!=None else 0)
no_climate_votes['CO2'] = no_climate_votes['description_text'].apply(lambda x: 1 if re.search(co2, x)!=None else 0)
no_climate_votes['grøn_omstilling'] = no_climate_votes['description_text'].apply(lambda x: 1 if re.search(grøn_omstilling, x)!=None else 0)
no_climate_votes['global_opvarmning'] = no_climate_votes['description_text'].apply(lambda x: 1 if re.search(global_opvarmning, x)!=None else 0)


In [10]:
p_false_negatives = no_climate_votes[(no_climate_votes.CO2 == 1) | (no_climate_votes.klimaforandringer==1) | (no_climate_votes.grøn_omstilling==1)]

p_false_negatives['description_text']
#There seem to be some duplicates (Worth looking into some data_cleaning here)
p_false_negatives.drop_duplicates(subset='description_text', inplace=True)
p_false_negatives.reset_index(inplace=True, drop=True)


/home/gugy/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [11]:
print(len(p_false_negatives))
p_false_negatives['false_negative'] = 0

58


/home/gugy/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [12]:
p_false_negatives['description_text'][57]

'Formålet med lovforslaget er at fremme udnyttelsen af overskudsvarme. Lovforslaget gennemfører initiativer, der har ophæng i Energiaftale af 29. juni 2018, Aftale om øget udnyttelse af overskudsvarme af 23. marts 2019, Klimaaftale for energi og industri mv. af 22. juni 2020  (S, V, DF, RV, SF, EL, KF, LA og ALT) og Opfølgende aftale om fremme af overskudsvarme af 7. september 2021 mellem samme partier. Lovforslaget indeholder tre hovedpunkter; prisregulering af overskudsvarme, herunder en bagatelgrænse for prisregulering, en energieffektiviseringsordning for overskudsvarme og afgiftsfritagelse for energieffektiv ekstern overskudsvarme.'

In [13]:
index_lst = [0, 1, 2, 6, 9, 13, 17, 18, 20, 26, 52, 53, 56, 57]
for i in index_lst:
    p_false_negatives.at[i, 'false_negative'] = 1

In [14]:
p_false_negatives[['description_text', 'false_negative']]

len(p_false_negatives[p_false_negatives['false_negative']==1])


14

In [15]:
view = p_false_negatives[p_false_negatives['false_negative']==1]



,index,vote_id,description,votes,votes_by_party,vote_caller,url,description_text,date,y_pred,klimaforandringer,CO2,grøn_omstilling,global_opvarmning,false_negative
0,78,"Afstemning nr. 270, 2004-05 (1. samling)","[Sagsgang:, Fremsat 17-11-2004, 1. behandlet /...","{'party': ['Venstre (V)', 'Uden for folketings...","{'Absent': ['14', '18', '7', '9', '6', '5', '3...","[Skatteminister, Kristian Jensen (V)]",https://www.ft.dk//samling/20041/afstemning/27...,Loven indeholder følgende elementer: - CO2-afg...,2004-12-20,non-climate,0,1,0,0,1
1,95,"Afstemning nr. 264, 2004-05 (1. samling)","[Sagsgang:, Fremsat 07-10-2004, 1. behandlet /...","{'party': ['Venstre (V)', 'Uden for folketings...","{'Absent': ['18', '17', '7', '9', '6', '5', '3...","[Skatteminister, Kristian Jensen (V)]",https://www.ft.dk//samling/20041/afstemning/26...,Loven indfører regler om den skattemæssige beh...,2004-12-20,non-climate,0,1,0,0,1
2,1469,"Afstemning nr. 503, 2006-07","[Sagsgang:, Fremsat 25-04-2007, 1. behandlet /...","{'party': ['Det Konservative Folkeparti (KF)',...","{'Absent': ['17', '17', '8', '7', '8', '2', '4...","[Skatteminister, Kristian Jensen (V)]",https://www.ft.dk//samling/20061/afstemning/50...,Loven medfører en omlægning af den samlede bil...,2007-06-06,non-climate,0,1,0,0,1
6,2309,"Afstemning nr. 466, 2008-09","[Sagsgang:, Fremsat 22-04-2009, 1. behandlet /...","{'party': ['Liberal Alliance (LA)', 'Dansk Fol...","{'Absent': ['13', '17', '7', '5', '7', '4', '2...","[Skatteminister, Kristian Jensen (V)]",https://www.ft.dk//samling/20081/afstemning/46...,Energiafgiftslovene ændres som led i udmøntnin...,2009-05-12,non-climate,1,1,0,0,1
9,2444,"Afstemning nr. 403, 2008-09","[Sagsgang:, Fremsat 04-02-2009, 1. behandlet /...","{'party': ['Liberal Alliance (LA)', 'Dansk Fol...","{'Absent': ['16', '18', '8', '7', '8', '4', '2...","[Skatteminister, Kristian Jensen (V)]",https://www.ft.dk//samling/20081/afstemning/40...,Loven omlægger forbrændingsafgiften af affald ...,2009-06-12,non-climate,0,1,0,0,1
13,2468,"Afstemning nr. 413, 2008-09","[Sagsgang:, Fremsat 22-04-2009, 1. behandlet /...","{'party': ['Liberal Alliance (LA)', 'Dansk Fol...","{'Absent': ['15', '19', '8', '7', '8', '4', '2...","[Skatteminister, Kristian Jensen (V)]",https://www.ft.dk//samling/20081/afstemning/41...,Loven var en udmøntning af forårspakke 2.0 på ...,2009-06-12,non-climate,0,1,0,0,1
17,2699,"Afstemning nr. 214, 2009-10","[Sagsgang:, Fremsat 25-11-2009, 1. behandlet /...","{'party': ['Liberal Alliance (LA)', 'Dansk Fol...","{'Absent': ['15', '17', '7', '9', '5', '4', '2...","[Skatteminister, Kristian Jensen (V)]",https://www.ft.dk//samling/20091/afstemning/21...,Med loven indføres der fradrag for udgifter ti...,2010-02-24,non-climate,1,1,0,0,1
18,2810,"Afstemning nr. 355, 2009-10","[Sagsgang:, Fremsat 10-02-2010, 1. behandlet /...","{'party': ['Liberal Alliance (LA)', 'Dansk Fol...","{'Absent': ['15', '16', '8', '8', '7', '6', '2...","[Økonomi- og erhvervsminister, Lene Espersen (...",https://www.ft.dk//samling/20091/afstemning/35...,Formålet med loven er at fastsætte nye regler ...,2010-05-26,non-climate,1,0,0,0,1
20,2982,"Afstemning nr. 135, 2010-11 (1. samling)","[Fremsat:, 14-12-2010, Forslag til vedtagelse:...","{'party': ['Liberal Alliance (LA)', 'Dansk Fol...","{'Absent': ['16', '17', '7', '8', '7', '4', '2...","[Jørgen S. Lundsgaard (KF), Lars Christian Lil...",https://www.ft.dk//samling/20101/afstemning/13...,"Folketinget konstaterer med tilfredshed, at ma...",2010-12-14,non-climate,1,0,0,0,1
26,3976,"Afstemning nr. 204, 2012-13","[Sagsgang:, Fremsat 16-11-2012, 1. behandlet /...","{'party': ['Dansk Folkeparti (DF)', 'Liberal A...","{'Absent': ['18', '16', '9', '5', '4', '4', '4...","[Skatteminister, Holger K. Nielsen (SF)]",https://www.ft.dk//samling/20121/afstemning/20...,Loven sanerer erhvervsstøtten på følgende områ...,2012-12-23,non-climate,0,1,0,0,1
